# Quasi-spherical coordinate transformation

We are looking for a coordinate transformation $(r, \varphi, \theta) = \Phi(x)$ in which the diffusion equation
$$
    \begin{aligned}
        \partial_t c          &= \Delta u   & \qquad\text{in}\qquad & \Omega \\
        \vec n \cdot \nabla u &= f          & \qquad\text{on}\qquad & \partial\Omega
    \end{aligned}
$$

takes the form

$$
    \begin{aligned}
        \partial_t c          &= \frac{1}{r^{d-1}} \partial_{r} (r^{d-1} \partial_{r} u)   & \qquad\text{in}\qquad & [0, R] \\
    \end{aligned}
$$

in dimension $d$.

In [ ]:
## General document imports
import dolfinx as dfx
from dolfinx.fem.petsc import LinearProblem

import gmsh

%matplotlib widget
import matplotlib.pyplot as plt

from mpi4py import MPI
from mpi4py.MPI import COMM_WORLD as comm

import numpy as np

import pyvista as pv

import ufl

### Open issues:

- increasing the order of the elements leads to negative radii even for the unit circle.
- which properties should the radial coordinate satisfy?
    - $r \in [0,1]$
    - $r = 1$ on $\partial\Omega$
    - why can we do the transformation and solve a 1d problem instead?
        - constant bcs
        - solve for the coordinate with the same operator

## Compute a generalization of the radius

First, we manufacture a problem and test how it behaves:
$$
    u = r^2 \qquad \Rightarrow\qquad \Delta u = 4 \qquad\text{on } \Omega
$$
and in particular
$$
    u = r^2 \qquad\text{on } \partial \Omega
$$

The weak form of the PDE:
$$
    -\int\limits_\Omega \nabla u \cdot \nabla v \, dV = -\int\limits_{\partial\Omega} v \underbrace{\nabla u \cdot \vec n}_{r} \, dS + \int\limits_\Omega 4 v \, dV
$$

### Circular mesh

As a workhorse, we create a circular mesh of which we compute the radius.
The boundary condition $\left. u \right|_{\partial \Omega} = 1$ determines the scaling.


In [ ]:
def create_unit_circle(comm, resolution):
    R = 1.

    gmsh.initialize()

    model = gmsh.model()

    model_name = "circle"

    gmsh.option.setNumber("Mesh.MeshSizeFactor", resolution)

    model.add(model_name)
    model.setCurrent(model_name)

    model.occ.addCircle(0, 0, 0, R, 1)
    model.occ.addCurveLoop([1], 2)
    gmsh.model.occ.addPlaneSurface([2],1)

    model.occ.synchronize()

    model.addPhysicalGroup(dim=1, tags=[1], tag=1)
    model.addPhysicalGroup(dim=2, tags=[1], tag=2)

    model.mesh.generate(dim=2)

    model.setCurrent(model_name)

    mesh, ct, ft = dfx.io.gmshio.model_to_mesh(gmsh.model, comm, rank=0)

    gmsh.finalize()

    return mesh

Since it won't change even when stretching the grid, we compute connectivity and find the boundary facets of the grid.

In [ ]:
mesh = create_unit_circle(comm, 0.1)

tdim = mesh.topology.dim
fdim = tdim - 1

mesh.topology.create_connectivity(fdim, tdim)

boundary_facets = dfx.mesh.exterior_facet_indices(mesh.topology)

For later reference, information on the grid size are computed:

In [ ]:
one = dfx.fem.Constant(mesh, 1.)

volume = dfx.fem.assemble_scalar(dfx.fem.form(one * ufl.dx))

surface = dfx.fem.assemble_scalar(dfx.fem.form(one * ufl.ds))

print(f"volume  = {volume:1.3e} (diff = {abs(volume - np.pi):1.3e})")
print(rf"surface = {surface:1.3e} (diff = {abs(surface - 2 * np.pi):1.3e})")

For the unit circle, we reconstruct the radial coordinate by integrating the problem
$$
    \Delta u = 4 \qquad\text{on}\quad \Omega
$$
with
$$
    \left. u \right|_{\partial\Omega} = r^2 = 1\,.
$$
The trivial solution is $u=r^2$, i.e., the radial coordinate is retrieved through $r = \sqrt u$.

In [ ]:
# The FEM space
V = dfx.fem.FunctionSpace(mesh, ("CG", 1))

# Construct the FEM form
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

# The FEM weak form
a = -ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = 4 * v * ufl.dx

# Boundary condition (dofs are extracted above).
bcs = [
    dfx.fem.dirichletbc(
        1., dfx.fem.locate_dofs_topological(V, fdim, boundary_facets), V)]

# Problem and solver setup.
problem = LinearProblem(a, L, bcs)

# The solution.
u = problem.solve()

In [ ]:
plotter = pv.Plotter()

grid = pv.UnstructuredGrid(*dfx.plot.vtk_mesh(V))

grid.point_data["u"] = u.x.array
grid.set_active_scalars("u")

grid = grid.warp_by_scalar('u')

plotter.add_mesh(grid, show_edges=False)

plotter.show()

In [ ]:
# Test the solution

eps = 1e-2

# We call the reconstructed coordinate `s`.
# A small eps is necessary to avoid zero-division since round-off errors may introduce small
# but negative numbers.
s = dfx.fem.Function(V)
s.interpolate(dfx.fem.Expression(
    ((u + eps))**0.5, V.element.interpolation_points()))

# The actual radial coordinate.
x, y, z = ufl.SpatialCoordinate(mesh)
r = (x**2 + y**2)**0.5

# The error.
error = dfx.fem.Function(V)
error.interpolate(dfx.fem.Expression(s - r, V.element.interpolation_points()))

error_max = np.abs(error.x.array).max()

print(f"Error : {error_max:1.3e}")


### Stretched circular mesh

The above problem definition relies on a consistent formulation the boundary conditions. In the spirit of a manufactured problem, we have taken the boundary values from the exact solution $u=^2$, that, obviously is affected by grid stretching. Subsequently, we want to delevop an algorithm with we can reconstrct the radial coordinate without prior knowlegde of the grid size.

In [ ]:
# store the unit circle mesh for later reference
unit_circle = mesh

mesh = create_unit_circle(comm, 0.1)

# Stretching the grid by multiplying the coordinate values by a factor of 2
mesh.geometry.x[:, :] *= 2

In [ ]:
V = dfx.fem.FunctionSpace(mesh, ("CG", 1))

# Construct the FEM form
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

a = -ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = 4 * v * ufl.dx

bcs = [
    dfx.fem.dirichletbc(
        1., dfx.fem.locate_dofs_topological(V, fdim, boundary_facets), V)]

# Problem and solver setup.
problem = LinearProblem(a, L, bcs)

# The solution.
u = problem.solve()

In [ ]:
plotter = pv.Plotter()

grid = pv.UnstructuredGrid(*dfx.plot.vtk_mesh(V))

grid.point_data["u"] = u.x.array
grid.set_active_scalars("u")

grid = grid.warp_by_scalar('u')

plotter.add_mesh(grid, show_edges=False)

plotter.show()

The solution, forced by the boundary condition $\left. u \right|_{\partial\Omega} = r^2 = 1$, now lies within $u(\Omega) \in [-3, 1]$. This necessarily follows from the rhs of $\Delta u = 4$. To obtain a meaningful physical coordinate, we might transform the solution s.t. $\tilde u \in [0,1]$.

In [ ]:
u_min_loc = u.x.array.min()
min_u = u.function_space.mesh.comm.allreduce(u_min_loc, op=MPI.MIN)

u_max_loc = u.x.array.max()
max_u = u.function_space.mesh.comm.allreduce(u_max_loc, op=MPI.MAX)

print(f"u_min = {min_u:1.3e}")
print(f"u_max = {max_u:1.3e}")

scale = (max_u - min_u)**0.5

u_tilde = (u - min_u) / (max_u - min_u)

# compute the radial coordinate
s = dfx.fem.Function(V)
s.interpolate(dfx.fem.Expression(
    u_tilde**0.5, V.element.interpolation_points()))

In [ ]:
plotter = pv.Plotter()

grid = pv.UnstructuredGrid(*dfx.plot.vtk_mesh(V))

grid.point_data["s"] = s.x.array
grid.set_active_scalars("s")

grid = grid.warp_by_scalar('s')

plotter.add_mesh(grid, show_edges=False)

plotter.show()

In [ ]:
# The actual radial coordinate.
x, y, z = ufl.SpatialCoordinate(mesh)
r = (x**2 + y**2)**0.5 / scale

# The error.
error = dfx.fem.Function(V)
error.interpolate(dfx.fem.Expression(abs(s - r), V.element.interpolation_points()))

max_loc_error = error.x.array.max()
max_error = mesh.comm.allreduce(max_loc_error, op=MPI.MAX)

print(f"error = {max_error:1.3e}")

With the procedure, we are able to reconstruct a scaled radial coordinate $s$ that satisfies the conditions:

- $s=1$ on $\partial\Omega$
- $0 \leq s<1$ in $\Omega \setminus \partial\Omega$
- $s^2$ is convex

### Distorted mesh

Now, we do the same as above but for a distorted mesh. To this end, first distort the mesh by a small perturbation.

In [ ]:
mesh = create_unit_circle(comm, 0.1)

x = mesh.geometry.x[:, :1]
y = mesh.geometry.x[:, 1:2]

r = np.sqrt(x**2 + y**2)

phi = np.arctan2(x, y)

e_r = np.array([
    np.cos(phi).squeeze(),
    np.sin(phi).squeeze()]).T

mesh.geometry.x[:, :2] = r * e_r * (2 + 0.2 * np.sin(3 * phi))

Again, we solve the problem $\Delta u = 4$ s.t. $u = s^2$.

In [ ]:
V = dfx.fem.FunctionSpace(mesh, ("CG", 1))

# Construct the form with Neumann conditions

u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

x, y, z = ufl.SpatialCoordinate(mesh)

a = -ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = 4 * v * ufl.dx

bcs = [
    dfx.fem.dirichletbc(
        1., dfx.fem.locate_dofs_topological(V, fdim, boundary_facets), V)]

from dolfinx.fem.petsc import LinearProblem

# a == L
problem = LinearProblem(a, L, bcs)

u = problem.solve()

In [ ]:
plotter = pv.Plotter()

grid = pv.UnstructuredGrid(*dfx.plot.vtk_mesh(V))

grid.point_data["u"] = u.x.array
grid.set_active_scalars("u")

plotter.add_mesh(grid, show_edges=True)

plotter.show()

In [ ]:
# Compute the radial coordinate s
u_min_loc = u.x.array.min()
min_u = u.function_space.mesh.comm.allreduce(u_min_loc, op=MPI.MIN)

u_max_loc = u.x.array.max()
max_u = u.function_space.mesh.comm.allreduce(u_max_loc, op=MPI.MAX)

print(f"u_min = {min_u:1.3e}")
print(f"u_max = {max_u:1.3e}")

scale = (max_u - min_u)**0.5

print(f"scale = {scale}")

u_tilde = (u - min_u) / (max_u - min_u) + 1e-8

# compute the radial coordinate
s = dfx.fem.Function(V)
s.interpolate(dfx.fem.Expression(
    u_tilde**0.5, V.element.interpolation_points()))

In [ ]:
plotter = pv.Plotter()

grid = pv.UnstructuredGrid(*dfx.plot.vtk_mesh(V))

grid.point_data["s"] = s.x.array
grid.set_active_scalars("s")

grid.warp_by_scalar('s', inplace=True)

plotter.add_mesh(grid, show_edges=True)

plotter.show()

In [ ]:
import dolfinx as dfx
import gmsh
import meshio
from mpi4py.MPI import COMM_WORLD as comm
import pygmsh

Lc1 = 0.01

L = 1.
H = 1.

geometry = pygmsh.geo.Geometry()

model = geometry.__enter__()

points = [
    model.add_point((0, 0, 0), Lc1),
    model.add_point((2 * L, 0, 0), Lc1),
    model.add_point((2 * L, 1 * H, 0), Lc1),
    model.add_point((0.5 * L, 0.5 * H, 0), 0.1 * Lc1),
    model.add_point((1 * L, 2 * H, 0), Lc1),
    model.add_point((0, 2 * H, 0), Lc1),
]

# Add lines between all points creating the L shape
lines = [model.add_line(points[i], points[i+1])
                 for i in range(-1, len(points)-1)]

# Create a line loop and plane surface for meshing
loop = model.add_curve_loop(lines)
plane_surface = model.add_plane_surface(loop)

# Call gmsh kernel before add physical entities
model.synchronize()

volume_marker = 6
model.add_physical([plane_surface], "Volume")
model.add_physical(lines, "Boundary")

geometry.generate_mesh(dim=2)
gmsh.write("mesh.msh")
gmsh.clear()
geometry.__exit__()

mesh_from_file = meshio.read("mesh.msh")

def create_mesh(mesh, cell_type, prune_z=False):
    cells = mesh.get_cells_type(cell_type)
    cell_data = mesh.get_cell_data("gmsh:physical", cell_type)
    points = mesh.points[:, :2] if prune_z else mesh.points
    out_mesh = meshio.Mesh(points=points, cells={cell_type: cells}, cell_data={
                           "name_to_read": [cell_data]})
    return out_mesh

triangle_mesh = create_mesh(mesh_from_file, "triangle", prune_z=True)
meshio.write("mesh.xdmf", triangle_mesh)

with dfx.io.XDMFFile(comm, "mesh.xdmf", 'r') as mesh_file:
    mesh = mesh_file.read_mesh(name="Grid")

In [ ]:
import pyvista as pv

plotter = pv.Plotter()

V = dfx.fem.FunctionSpace(mesh, ("CG", 1))

grid = pv.UnstructuredGrid(*dfx.plot.vtk_mesh(V))

plotter.add_mesh(grid, show_edges=True)

plotter.show()

In [ ]:
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

a = -ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = 4 * v * ufl.dx

tdim = mesh.topology.dim
fdim = tdim - 1

mesh.topology.create_connectivity(fdim, tdim)

boundary_facets = dfx.mesh.exterior_facet_indices(mesh.topology)

bcs = [
    dfx.fem.dirichletbc(
        1., dfx.fem.locate_dofs_topological(V, fdim, boundary_facets), V)]

from dolfinx.fem.petsc import LinearProblem

# a == L
problem = LinearProblem(a, L, bcs)

u = problem.solve()

In [ ]:
# Compute the radial coordinate s
u_min_loc = u.x.array.min()
min_u = u.function_space.mesh.comm.allreduce(u_min_loc, op=MPI.MIN)

u_max_loc = u.x.array.max()
max_u = u.function_space.mesh.comm.allreduce(u_max_loc, op=MPI.MAX)

print(f"u_min = {min_u:1.3e}")
print(f"u_max = {max_u:1.3e}")

scale = (max_u - min_u)**0.5

print(f"scale = {scale}")

u_tilde = (u - min_u) / (max_u - min_u) + 1e-8

# compute the radial coordinate
s = dfx.fem.Function(V)
s.interpolate(dfx.fem.Expression(
    u_tilde**0.5, V.element.interpolation_points()))

In [ ]:
plotter = pv.Plotter()

grid = pv.UnstructuredGrid(*dfx.plot.vtk_mesh(V))

grid.point_data["s"] = s.x.array
grid.set_active_scalars("s")

grid.warp_by_scalar('s', inplace=True)

plotter.add_mesh(grid, show_edges=True)

plotter.show()

## Use the quasi-radial coordinate

We want to use the quasi-radial coordinate $s$ computed previously to solve a quasi-symmetric boundary-value problem.

Thus, we want so solve the problem

$$
    \Delta u = f \qquad\text{on}\quad \partial \Omega,,
$$
where $f = f(s)$ is a function purely dependent on the quasi-radial coordinate. We choose $f = \exp(-s^2)$.

First, we solve on the 2D grid:

In [ ]:
# The right-hand side source.
f = dfx.fem.Function(V)

f.interpolate(dfx.fem.Expression(
    ufl.sin(-s**2), V.element.interpolation_points()))

u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

a = -ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = f * v * ufl.dx

bcs = [
    dfx.fem.dirichletbc(
        np.pi, dfx.fem.locate_dofs_topological(V, fdim, boundary_facets), V)]

# a == L
problem = LinearProblem(a, L, bcs)

u_2d = problem.solve()

In [ ]:
plotter = pv.Plotter()

grid = pv.UnstructuredGrid(*dfx.plot.vtk_mesh(V))

grid.point_data["u"] = u_2d.x.array
grid.set_active_scalars("u")

plotter.add_mesh(grid, show_edges=True)

plotter.show()

In [ ]:
mesh_1d = dfx.mesh.create_interval(MPI.COMM_WORLD, 128, (0, 1))

V_1d = dfx.fem.FunctionSpace(mesh_1d, ("CG", 1))

r = ufl.SpatialCoordinate(mesh_1d)[0]

# It is important to scale the 1D equation since we use normalize coordinates
f = ufl.sin(-r**2) * scale**2

u = ufl.TrialFunction(V_1d)
v = ufl.TestFunction(V_1d)

a = - r * ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = r * f * v * ufl.dx

def outer(x):
    return np.isclose(x[0], 1)

dofs = dfx.fem.locate_dofs_geometrical(V_1d, outer)

bcs = [dfx.fem.dirichletbc(np.pi, dofs, V_1d)]

# a == L
problem = LinearProblem(a, L, bcs)

u_1d = problem.solve()

# Plot the 1D solution
plt.figure()

plt.plot(mesh_1d.geometry.x[:, 0], u_1d.x.array)

plt.show()

In [ ]:
# Interpolate the solution to the 2D grid

import scipy as sp

poly = sp.interpolate.interp1d(mesh_1d.geometry.x[:, 0], u_1d.x.array, fill_value="extrapolate")

u_interp = dfx.fem.Function(V)
u_interp.x.array[:] = poly(s.x.array[:])

In [ ]:
plotter = pv.Plotter()

grid = pv.UnstructuredGrid(*dfx.plot.vtk_mesh(V))

grid.point_data["u"] = u_interp.x.array - u_2d.x.array
grid.set_active_scalars("u")

grid.point_data["s"] = s.x.array

grid = grid.warp_by_scalar('u')

plotter.add_mesh(grid, show_edges=True)

plotter.show()

## Use 3D meshes

We use one of Manuel's meshes to solve the Poisson equation in 3D.
Simultaneously, we solve in 1D using the quasi-radial coordinate according to the proposed algorithm.

In [ ]:
from pyMoBiMP.gmsh_utils import dfx_spherical_mesh

mesh, _, _ = dfx_spherical_mesh(comm, resolution=0.1, optimize=False)

V = dfx.fem.FunctionSpace(mesh, ("CG", 1))

# Construct the FEM form
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

a = -ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = 6 * v * ufl.dx

tdim = mesh.topology.dim
fdim = tdim - 1

mesh.topology.create_connectivity(fdim, tdim)

boundary_facets = dfx.mesh.exterior_facet_indices(mesh.topology)

bcs = [
    dfx.fem.dirichletbc(
        1., dfx.fem.locate_dofs_topological(V, fdim, boundary_facets), V)]

# Problem and solver setup.
problem = LinearProblem(a, L, bcs)

# The solution.
u = problem.solve()

In [ ]:
# Compute the radial coordinate s
u_min_loc = u.x.array.min()
min_u = u.function_space.mesh.comm.allreduce(u_min_loc, op=MPI.MIN)

u_max_loc = u.x.array.max()
max_u = u.function_space.mesh.comm.allreduce(u_max_loc, op=MPI.MAX)

print(f"u_min = {min_u:1.3e}")
print(f"u_max = {max_u:1.3e}")

scale = (max_u - min_u)**0.5

print(f"scale = {scale}")

u_tilde = (u - min_u) / (max_u - min_u) + 1e-8

# compute the radial coordinate
s = dfx.fem.Function(V)
s.interpolate(dfx.fem.Expression(
    u_tilde**0.5, V.element.interpolation_points()))

In [ ]:
topology, cell_types, x = dfx.plot.vtk_mesh(V)
grid = pv.UnstructuredGrid(topology, cell_types, x)

grid["u"] = s.x.array

clip_box = np.array([0., 20., 0., 20., -20, 20.])

clipped = grid.clip_box(clip_box, crinkle=False)

plotter = pv.Plotter()

plotter.add_mesh(clipped, show_edges=True, show_scalar_bar=True)
# plotter.add_mesh(grid, style="wireframe", color="blue")

plotter.show()

In [ ]:
mesh_1d = dfx.mesh.create_interval(MPI.COMM_WORLD, 128, (0, 1))

V_1d = dfx.fem.FunctionSpace(mesh_1d, ("CG", 1))

r = ufl.SpatialCoordinate(mesh_1d)[0]

# It is important to scale the 1D equation since we use normalize coordinates
f = ufl.exp(-r**2) * scale**2

u = ufl.TrialFunction(V_1d)
v = ufl.TestFunction(V_1d)

a = - r * ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = r * f * v * ufl.dx

def outer(x):
    return np.isclose(x[0], 1)

dofs = dfx.fem.locate_dofs_geometrical(V_1d, outer)

bcs = [dfx.fem.dirichletbc(np.pi, dofs, V_1d)]

# a == L
problem = LinearProblem(a, L, bcs)

u_1d = problem.solve()

# Plot the 1D solution
plt.figure()

plt.plot(mesh_1d.geometry.x[:, 0], u_1d.x.array)

plt.show()

In [ ]:
base_dir = "../Meshes/Particles/NonSpherical/"
file = "singleParticle_Tom2_vol"

vertices = np.loadtxt(base_dir + file + ".node")
cells = np.loadtxt(base_dir + file + ".elem", dtype=np.int32)

gdim = 3
shape = "tetrahedron"
degree = 1

cell = ufl.Cell(shape, geometric_dimension=gdim)
domain = ufl.Mesh(ufl.VectorElement("Lagrange", cell, degree))

mesh = dfx.mesh.create_mesh(MPI.COMM_WORLD, cells[:, :4] - 1, vertices, domain)

In [ ]:
plotter = pv.Plotter()

V = dfx.fem.FunctionSpace(mesh, ("CG", 1))

grid = pv.UnstructuredGrid(*dfx.plot.vtk_mesh(V))

plotter.add_mesh(grid, show_edges=True)

plotter.show()

First, we solve for the 3D quasi-radial coordinate.

In [ ]:
V = dfx.fem.FunctionSpace(mesh, ("CG", 1))

# Construct the FEM form
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

a = -ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = 6 * v * ufl.dx

tdim = mesh.topology.dim
fdim = tdim - 1

mesh.topology.create_connectivity(fdim, tdim)

boundary_facets = dfx.mesh.exterior_facet_indices(mesh.topology)

bcs = [
    dfx.fem.dirichletbc(
        1., dfx.fem.locate_dofs_topological(V, fdim, boundary_facets), V)]

# Problem and solver setup.
problem = LinearProblem(a, L, bcs)

# The solution.
u = problem.solve()

In [ ]:
# Compute the radial coordinate s
u_min_loc = u.x.array.min()
min_u = u.function_space.mesh.comm.allreduce(u_min_loc, op=MPI.MIN)

u_max_loc = u.x.array.max()
max_u = u.function_space.mesh.comm.allreduce(u_max_loc, op=MPI.MAX)

print(f"u_min = {min_u:1.3e}")
print(f"u_max = {max_u:1.3e}")

scale = (max_u - min_u)**0.5

print(f"scale = {scale}")

u_tilde = (u - min_u) / (max_u - min_u) + 1e-8

# compute the radial coordinate
s = dfx.fem.Function(V)
s.interpolate(dfx.fem.Expression(
    u_tilde**0.5, V.element.interpolation_points()))

In [ ]:
topology, cell_types, x = dfx.plot.vtk_mesh(V)
grid = pv.UnstructuredGrid(topology, cell_types, x)

grid["u"] = s.x.array

clip_box = np.array([0., 20., 0., 20., -20, 20.])

clipped = grid.clip_box(clip_box, crinkle=True)

plotter = pv.Plotter()

plotter.add_mesh(clipped, show_edges=True, show_scalar_bar=True)
# plotter.add_mesh(grid, style="wireframe", color="blue")

plotter.show()

Here, we construct the 3D version of the above Poisson problem.

In [ ]:
# The right-hand side source.
f = dfx.fem.Function(V)

f.interpolate(dfx.fem.Expression(
    ufl.exp(-s**2), V.element.interpolation_points()))

u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

a = -ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = f * v * ufl.dx

bcs = [
    dfx.fem.dirichletbc(
        np.pi, dfx.fem.locate_dofs_topological(V, fdim, boundary_facets), V)]

# a == L
problem = LinearProblem(a, L, bcs)

u_3d = problem.solve()

In [ ]:
topology, cell_types, x = dfx.plot.vtk_mesh(V)
grid = pv.UnstructuredGrid(topology, cell_types, x)

grid["u"] = u_3d.x.array

clip_box = np.array([0., 20., 0., 20., -20, 20.])

clipped = grid.clip_box(clip_box, crinkle=True)

plotter = pv.Plotter()

plotter.add_mesh(clipped, show_edges=True, show_scalar_bar=True)
# plotter.add_mesh(grid, style="wireframe", color="blue")

plotter.show()

And we constuct the 1D problem in the quasi-radial coordinates according to the present scaling.

In [ ]:
mesh_1d = dfx.mesh.create_interval(MPI.COMM_WORLD, 128, (0, 1))

V_1d = dfx.fem.FunctionSpace(mesh_1d, ("CG", 1))

r = ufl.SpatialCoordinate(mesh_1d)[0]

# It is important to scale the 1D equation since we use normalize coordinates
f = ufl.exp(-r**2) * scale**2

u = ufl.TrialFunction(V_1d)
v = ufl.TestFunction(V_1d)

a = - r**2 * ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = r**2 * f * v * ufl.dx

def outer(x):
    return np.isclose(x[0], 1)

dofs = dfx.fem.locate_dofs_geometrical(V_1d, outer)

bcs = [dfx.fem.dirichletbc(np.pi, dofs, V_1d)]

# a == L
problem = LinearProblem(a, L, bcs)

u_1d = problem.solve()

# Plot the 1D solution
plt.figure()

plt.plot(mesh_1d.geometry.x[:, 0], u_1d.x.array)

plt.show()

Shown below is the 3D representation of the quasi-1D solution.

In [ ]:
poly = sp.interpolate.interp1d(mesh_1d.geometry.x[:, 0], u_1d.x.array, fill_value="extrapolate")

u_interp = dfx.fem.Function(V)
u_interp.x.array[:] = poly(s.x.array[:])

In [ ]:
topology, cell_types, x = dfx.plot.vtk_mesh(V)
grid = pv.UnstructuredGrid(topology, cell_types, x)

grid["u"] = u_interp.x.array

clip_box = np.array([0., 20., 0., 20., -20, 20.])

clipped = grid.clip_box(clip_box, crinkle=True)

plotter = pv.Plotter()

plotter.add_mesh(clipped, show_edges=True, show_scalar_bar=True)
# plotter.add_mesh(grid, style="wireframe", color="blue")

plotter.show()

Also show the difference between the two approaches.

In [ ]:
plotter = pv.Plotter()

grid = pv.UnstructuredGrid(*dfx.plot.vtk_mesh(V))

grid.point_data["u"] = u_interp.x.array - u_3d.x.array
grid.set_active_scalars("u")

clip_box = np.array([0., 20., 0., 20., -20, 20.])

clipped = grid.clip_box(clip_box, crinkle=True)

plotter.add_mesh(clipped, show_edges=True)

plotter.show()